<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorith_VQA_VQE(PySCFOnly)_mkbahk_20251212.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.5 MB/s eta 0:00:00


In [2]:
import numpy as np
from pyscf import gto, scf, mp, ci, cc

In [15]:
# 물분자 구조 정의 (단위: Angstrom)
mol = gto.Mole()

#물분자 구조
#mol.atom = '''
#    O  0.0000  0.0000  0.1173
#    H  0.0000  0.7572 -0.4692
#    H  0.0000 -0.7572 -0.4692
#'''

#에탄올 (Ethanol, C₂H₅OH) – 탄소(C) + 수소(H) + 산소(O)
#3-종류 원소 (staggered conformation).

#mol.atom = """
#C   0.000000   0.000000   0.739000
#C   0.000000   0.000000  -0.811000
#O   0.000000   1.395000   1.139000
#H   0.000000  -0.951000   1.309000
#H   0.000000   0.951000   1.309000
#H   0.823000   0.000000  -1.381000
#H  -0.823000   0.000000  -1.381000
#H   0.000000   0.000000  -1.381000
#H   0.000000   1.395000   2.139000
#"""

mol.atom = """
# [Ru(bpy)3]2+ 양이온 (Ru 중심)
Ru   0.023388   0.011203   0.043858

# 첫 번째 bpy 리간드 (N1, C들, H들)
N  -6.996974   1.888644   1.766836
C  -7.246019   3.224446   1.620389
C  -6.434692   4.191354   2.225471
C  -5.366033   3.773434   3.015910
C  -5.139246   2.410992   3.200836
C  -5.973694   1.495662   2.557680
H  -6.645818   5.255281   2.092446
H  -4.721777   4.514441   3.491641
H  -4.330167   2.051166   3.837262
H  -5.847213   0.757000   2.100000  # 추가 H 근사

# 두 번째 bpy 리간드 (대칭 배치, 회전 적용)
N   0.023388   0.011203  -0.043858  # 대칭
C   0.273000  -1.888644  -1.766836
C   0.523000  -3.224446  -1.620389
C   1.335000  -4.191354  -2.225471
C   2.403000  -3.773434  -3.015910
C   2.630000  -2.410992  -3.200836
C   2.396000  -1.495662  -2.557680
H   0.927000  -5.255281  -2.092446
H   3.050000  -4.514441  -3.491641
H   3.440000  -2.051166  -3.837262
H   2.722000  -0.757000  -2.100000

# 세 번째 bpy 리간드 (대칭 배치)
N   6.973586  -1.877441  -1.722978
C   6.723541  -3.213282  -1.577525
C   7.535000  -4.179890  -2.182051
C   8.604000  -3.761882  -2.972595
C   8.831000  -2.399530  -3.157439
C   7.997000  -1.484191  -2.514749
H   7.325000  -5.239923  -2.048520
H   9.249000  -4.502883  -3.452859
H   9.559000  -1.999000  -3.789000
H   8.283000  -0.743000  -2.057000

# 두 개의 PF6- 이온 (대칭 배치, Ru 근처)
P   5.000000   0.000000   0.000000
F   5.000000   1.200000   0.000000
F   5.000000  -1.200000   0.000000
F   6.200000   0.000000   0.000000
F   3.800000   0.000000   0.000000
F   5.000000   0.000000   1.200000
F   5.000000   0.000000  -1.200000
P  -5.000000   0.000000   0.000000
F  -5.000000   1.200000   0.000000
F  -5.000000  -1.200000   0.000000
F  -6.200000   0.000000   0.000000
F  -3.800000   0.000000   0.000000
F  -5.000000   0.000000   1.200000
F  -5.000000   0.000000  -1.200000

# 추가 이온 (전하 균형 및 10종 원소 확보, 결정 격자 근사 위치)
K   8.000000   0.000000   0.000000
Cl  10.000000  0.000000   0.000000
Br  12.000000  0.000000   0.000000
I   14.000000  0.000000   0.000000
"""

#mol.basis = '6-31g'
mol.basis = 'sto-3g'
mol.unit = 'angstrom'
mol.spin = 1 # 435개의 전자는 홀수이므로 spin=0(단일항)과 일치하지 않습니다. spin=1(이중항)으로 설정합니다.
mol.build()

print("=" * 60)
print("계산용 분자 바닥상태 에너지 계산")
print("=" * 60)
print(f"기저 함수: {mol.basis}")
print(f"전자 수: {mol.nelectron}")
print(f"원자 수: {mol.natm}")
print("=" * 60)


계산용 분자 바닥상태 에너지 계산
기저 함수: sto-3g
전자 수: 435
원자 수: 50


In [ ]:
# 1. Hartree-Fock (HF) 계산
print("\n[1] Hartree-Fock (HF) 계산")
mf = scf.RHF(mol)
hf_energy = mf.kernel()
print(f"HF 에너지: {hf_energy:.10f} Hartree")


[1] Hartree-Fock (HF) 계산


In [5]:
# 2. MP2 (Møller-Plesset 2차 섭동론) 계산
print("\n[2] MP2 계산")
mp2 = mp.MP2(mf)
mp2_energy = mp2.kernel()[0]
print(f"MP2 상관 에너지: {mp2_energy:.10f} Hartree")
print(f"MP2 총 에너지: {hf_energy + mp2_energy:.10f} Hartree")


[2] MP2 계산
E(RMP2) = -151.948070484214  E_corr = -0.28959190742564
E(SCS-RMP2) = -151.949390259993  E_corr = -0.290911683204967
MP2 상관 에너지: -0.2895919074 Hartree
MP2 총 에너지: -151.9480704842 Hartree


In [6]:
# 3. CCSD (Coupled Cluster Singles and Doubles) 계산
print("\n[3] CCSD 계산")
ccsd = cc.CCSD(mf)
ccsd_energy = ccsd.kernel()[0]
print(f"CCSD 상관 에너지: {ccsd_energy:.10f} Hartree")
print(f"CCSD 총 에너지: {hf_energy + ccsd_energy:.10f} Hartree")


[3] CCSD 계산
E(CCSD) = -151.9819852723944  E_corr = -0.3235066956062641
CCSD 상관 에너지: -0.3235066956 Hartree
CCSD 총 에너지: -151.9819852724 Hartree


In [7]:
# 4. CISD (Configuration Interaction Singles and Doubles) 계산
print("\n[4] CISD 계산")
cisd = ci.CISD(mf)
cisd_energy = cisd.kernel()[0]
print(f"CISD 상관 에너지: {cisd_energy:.10f} Hartree")
print(f"CISD 총 에너지: {hf_energy + cisd_energy:.10f} Hartree")


[4] CISD 계산
E(RCISD) = -151.9495901437925  E_corr = -0.2911115670043314
CISD 상관 에너지: -0.2911115670 Hartree
CISD 총 에너지: -151.9495901438 Hartree


In [8]:
# Numpy를 이용한 에너지 비교 및 분석
print("\n" + "=" * 60)
print("에너지 비교 및 분석 (Numpy 사용)")
print("=" * 60)

# 에너지 값을 배열로 저장
methods = np.array(['HF', 'MP2', 'CCSD', 'CISD'])
energies = np.array([
    hf_energy,
    hf_energy + mp2_energy,
    hf_energy + ccsd_energy,
    hf_energy + cisd_energy
])

# HF 에너지를 기준으로 한 상대 에너지 (kcal/mol)
hartree_to_kcal = 627.509474  # 변환 계수
relative_energies = (energies - energies[0]) * hartree_to_kcal

print("\n총 에너지 비교:")
for i, method in enumerate(methods):
    print(f"{method:6s}: {energies[i]:15.10f} Hartree")
###for

print("\n상대 에너지 (HF 기준, kcal/mol):")
for i, method in enumerate(methods):
    print(f"{method:6s}: {relative_energies[i]:10.4f} kcal/mol")
###for



에너지 비교 및 분석 (Numpy 사용)

총 에너지 비교:
HF    : -151.6584785768 Hartree
MP2   : -151.9480704842 Hartree
CCSD  : -151.9819852724 Hartree
CISD  : -151.9495901438 Hartree

상대 에너지 (HF 기준, kcal/mol):
HF    :     0.0000 kcal/mol
MP2   :  -181.7217 kcal/mol
CCSD  :  -203.0035 kcal/mol
CISD  :  -182.6753 kcal/mol


In [9]:
# 통계 분석
print("\n통계 분석 (Numpy):")
print(f"평균 에너지: {np.mean(energies):.10f} Hartree")
print(f"표준편차: {np.std(energies):.10f} Hartree")
print(f"최소 에너지: {np.min(energies):.10f} Hartree ({methods[np.argmin(energies)]})")
print(f"최대 에너지: {np.max(energies):.10f} Hartree ({methods[np.argmax(energies)]})")


통계 분석 (Numpy):
평균 에너지: -151.8845311193 Hartree
표준편차: 0.1312126056 Hartree
최소 에너지: -151.9819852724 Hartree (CCSD)
최대 에너지: -151.6584785768 Hartree (HF)


In [10]:
# 상관 에너지 비교
correlation_energies = np.array([0, mp2_energy, ccsd_energy, cisd_energy])
print("\n상관 에너지 회복률 (MP2 대비):")
for i in range(1, len(methods)):
    if i == 1:
        print(f"{methods[i]:6s}: 100.00% (기준)")
    else:
        recovery = (correlation_energies[i] / mp2_energy) * 100
        print(f"{methods[i]:6s}: {recovery:6.2f}%")
    ###if
###for


상관 에너지 회복률 (MP2 대비):
MP2   : 100.00% (기준)
CCSD  : 111.71%
CISD  : 100.52%


In [11]:
# 에너지 차이 행렬 (kcal/mol)
print("\n에너지 차이 행렬 (kcal/mol):")
print("       ", "".join([f"{m:>10s}" for m in methods]))
energy_diff_matrix = np.zeros((len(energies), len(energies)))
for i in range(len(energies)):
    diff_str = f"{methods[i]:6s} "
    for j in range(len(energies)):
        energy_diff_matrix[i, j] = (energies[i] - energies[j]) * hartree_to_kcal
        diff_str += f"{energy_diff_matrix[i, j]:10.4f}"
    ###for
    print(diff_str)
###for

print("\n" + "=" * 60)
print("계산 완료!")
print("=" * 60)


에너지 차이 행렬 (kcal/mol):
                HF       MP2      CCSD      CISD
HF         0.0000  181.7217  203.0035  182.6753
MP2     -181.7217    0.0000   21.2819    0.9536
CCSD    -203.0035  -21.2819    0.0000  -20.3283
CISD    -182.6753   -0.9536   20.3283    0.0000

계산 완료!
